In [1]:
import pandas as pd
import numpy as np
import re
import json
import string
import spacy

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English


In [2]:
punctuations = string.punctuation
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()

In [3]:

def spacy_tokenizer(sent):
    tokens = parser(sent)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations]
    return tokens

class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}


def clean_text(text):
  
    return text.strip().lower()

In [4]:
with open("_data/reviews.json", encoding='utf-8', errors='ignore') as json_data:
     data = json.load(json_data, strict=False)

In [5]:
fix = [v for v in data['reviews']]

In [6]:
reviews=pd.DataFrame.from_dict(fix, orient='columns', dtype=None, columns=None)

In [7]:
#reviews.head()

reviews.info()

#reviews.stars.value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659554 entries, 0 to 659553
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   review_id    659554 non-null  object 
 1   user_id      659554 non-null  object 
 2   business_id  659554 non-null  object 
 3   stars        659554 non-null  float64
 4   useful       659554 non-null  int64  
 5   funny        659554 non-null  int64  
 6   cool         659554 non-null  int64  
 7   text         659554 non-null  object 
 8   date         659554 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 45.3+ MB


In [8]:
reviews = reviews[reviews.stars != 3]

In [9]:
reviews['sentiment'] = np.where(reviews.stars>3, 1, 0)

In [10]:
reviews.stars.value_counts()

5.0    290558
4.0    144186
1.0     99326
2.0     53249
Name: stars, dtype: int64

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,sentiment
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,0
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,1
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03,1
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,1
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,0
...,...,...,...,...,...,...,...,...,...,...
659548,kv7eFrI2IUkQAU8PbP1EoA,T1XUUXIEU5uj3NmdWAdIlA,3-dR6RJvTHdkZEHMmB8OeQ,4.0,0,0,0,Nothing bad about the food. Quick service. Pri...,2012-02-01 05:27:31,1
659549,72S7F4topR8dYvrQjhJGXw,KjpcYyobG6NwPLqZWoNKkA,BBTkC1Z3ZnEE916SkCONIg,5.0,1,0,0,Finally a place to eat something other than fr...,2011-02-13 22:19:06,1
659550,GFJblUSfCfcjGJsI8_yPYQ,8kZsE4fLDufW20JKfEOnYA,fYjSmncFNMdDdBK3uJpKEQ,5.0,10,7,6,"I'm not close to Jamaican, nor am i an expert ...",2012-01-03 22:37:32,1
659552,MAFUnkhpO1NC03Qyn_1vbQ,c9zDQl4vQg41Pxmc5TpWWQ,nxCRx5_US4XLxufkF6-Tlw,5.0,1,0,0,30 July 2013\n\nGreat customer service by the ...,2013-08-04 01:38:14,1
